In [ ]:
import numpy as np
import pandas as pd

import plotly.express as px
import plotly.io as pio

pio.templates.default="simple_white"

from progtools.preprocessing import RawFlightData
from progtools.visuals import plot_df_color_per_unit, matrix_plot

In [ ]:
#read the raw flight data in
filename = "data/N-CMAPSS_DS05.h5"
raw_data_read = RawFlightData(filename=filename)

#create data frames for flight, sensor and auxiliary data [respectively]
df_W = raw_data_read.all_flight_data()
df_X_s = raw_data_read.all_sensor_data()
df_A = raw_data_read.all_aux_data()

In [ ]:
#split out the healthy and unhealthy samples
df_W_healthy=df_W[df_A["hs"]==1]
df_W_unhealthy=df_W[df_A["hs"]==0]

df_X_s_healthy=df_X_s[df_A["hs"]==1]
df_X_s_unhealthy=df_X_s[df_A["hs"]==0]

In [ ]:
#create combined dataframes for correlation analysis
df_W_X_s_healthy = df_W_healthy.join(df_X_s_healthy)
df_W_X_s_unhealthy = df_W_unhealthy.join(df_X_s_unhealthy)

In [ ]:
#determine units in development dataset
raw_data_read.dev_aux_data().unit.unique()

In [ ]:
#determine units in test dataset
raw_data_read.test_aux_data().unit.unique()

In [ ]:
#determine length of development dataset
raw_data_read.dev_aux_data().shape[0]

In [ ]:
#determine length of test dataset
raw_data_read.test_aux_data().shape[0]

In [ ]:
#print out the maximum number of cycles [flights] for each unit [engine]
for unit in df_A["unit"].unique():
    cyc = df_A[df_A["unit"]==unit]["cycle"].max()
    print(f"Maximum number of cycles for unit {unit:.0f}: {cyc:.0f}")

In [ ]:
#create a new column in flight descriptor dataframe mirroring "unit" from aux. dataframe
df_W["unit"]=df_A["unit"].values

#isolate a specific cycle for a specific unit in a new dataframe
df_W_u = df_W.loc[(df_A.unit==5) & (df_A.cycle == 5)]

#reset the index so it can be used as time
df_W_u.reset_index(inplace=True, drop=True)

#create a series of labels for subplots
labels = ["Altitude [ft]", "Mach Number [-]", "Throttle Resolver Angle [%]", "Temperature at fan inlet (T2) [°R]"]

#create a plot of flight descriptors versus time for isolated unit and cycle
plot_df_color_per_unit(df_W_u, df_W_u.columns.drop(["unit"]), labels, size=7, labelsize=14)

In [ ]:
#create a new column in physical sensor dataframe mirroring "unit" from aux. dataframe
df_X_s["unit"]=df_A["unit"].values

#isolate a specific cycle for a specific unit in a new dataframe
df_X_s_u_c = df_X_s.loc[(df_A.unit == 5) & (df_A.cycle == 5)]

#reset the index so it can be used as time
df_X_s_u_c.reset_index(inplace=True, drop=True)

#create a plot of each physical sensor versus time for unit and cycle of interest
plot_df_color_per_unit(df_X_s_u_c, df_X_s.columns.drop(["unit"]), df_X_s.columns.drop(["unit"]),size=10)

In [ ]:
#create a plot of health state versus cycle for a specific unit

plot_df_color_per_unit(df_A[df_A["unit"]==5], ["hs"], [r"$h_s$ [-]"], option='cycle')

In [ ]:
#create seperate correlation dataframes for healthy and unhealthy health states of flight descriptors; using Spearman R correlation

corr_W_healthy=df_W_healthy.corr(method="spearman")
corr_W_unhealthy=df_W_unhealthy.corr(method="spearman")

In [ ]:
#create a correlation matrix plot for healthy state for flight descriptors

matrix_plot(corr_W_healthy)

In [ ]:
#create a correlation matrix plot for unhealthy state for flight descriptors

matrix_plot(corr_W_unhealthy)

In [ ]:
#create seperate correlation dataframes for healthy and unhealthy health states of physical sensors; using Spearman R correlation

corr_X_s_healthy=df_X_s_healthy.corr(method="spearman")
corr_X_s_unhealthy=df_X_s_unhealthy.corr(method="spearman")

In [ ]:
#create matrix correlation plot for physical sensors during healthy state
matrix_plot(corr_X_s_healthy)

In [ ]:
#create seperate correlation dataframes for both health states for flight descriptors and physical sensors
#using Spearman R correlation

corr_W_X_s_healthy=df_W_X_s_healthy.corr(method="spearman")
corr_W_X_s_unhealthy=df_W_X_s_unhealthy.corr(method="spearman")

In [ ]:
#create a correlation matrix plot for healthy state for flight descriptors versus physical sensors

matrix_plot(corr_W_X_s_healthy.loc[df_W.columns.drop(["unit"]),df_X_s.columns.drop(["unit"])])

In [ ]:
#create a correlation matrix plot for unhealthy state for flight descriptors versus physical sensors

matrix_plot(corr_W_X_s_unhealthy.loc[df_W.columns.drop(["unit"]),df_X_s.columns.drop(["unit"])])

In [ ]:
#create a differential correlation dataframe of healthy and unhealthy flight descriptors and physical sensors

corr_W_X_s_delta=corr_W_X_s_healthy-corr_W_X_s_unhealthy

In [ ]:
#create a correlation matrix plot for delta in healhty state for flight descriptors versus physical sensors

matrix_plot(corr_W_X_s_delta.loc[df_W.columns.drop(["unit"]),df_X_s.columns.drop(["unit"])],plot_range=[-0.04,0.04])